# Train RNN/LSTM on Light Curve Data
This notebook trains RNN or LSTM models on light curve data for transient classification

In [ ]:
# Clone repository
!git clone https://github.com/YOUR_USERNAME/CS230-Detecting-Transients-in-LSST-Observatory-Data.git
%cd CS230-Detecting-Transients-in-LSST-Observatory-Data

In [ ]:
# Check GPU availability
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

In [ ]:
# Upload your processed_training.csv to Colab
# Option 1: Upload from local
from google.colab import files
uploaded = files.upload()
!mkdir -p data
!mv processed_training.csv data/

# Option 2: Download from Google Drive (uncomment if using this method)
# from google.colab import drive
# drive.mount('/content/drive')
# !cp /content/drive/MyDrive/processed_training.csv data/

In [ ]:
# Verify data is loaded
import pandas as pd
df = pd.read_csv('data/processed_training.csv')
print(f"Data shape: {df.shape}")
print(f"Columns: {list(df.columns)}")
print(f"Unique objects: {df['object_id'].nunique()}")
print(f"Unique targets: {sorted(df['target'].unique())}")

## Train RNN Model

In [ ]:
# Train RNN
!python train_with_data.py \
    --data_path data/processed_training.csv \
    --model rnn \
    --epochs 20 \
    --batch_size 32 \
    --hidden_size 128 \
    --num_layers 3 \
    --lr 0.001 \
    --max_length 200 \
    --save_model

## Train LSTM Model

In [ ]:
# Train LSTM
!python train_with_data.py \
    --data_path data/processed_training.csv \
    --model lstm \
    --epochs 20 \
    --batch_size 32 \
    --hidden_size 128 \
    --num_layers 3 \
    --lr 0.001 \
    --max_length 200 \
    --save_model

## Download Trained Models

In [ ]:
# Download trained models to your local machine
from google.colab import files
files.download('rnn_model.pth')
files.download('lstm_model.pth')

## Test Model (Optional)

In [ ]:
# Quick test of trained model
import torch
from models.rnn import LightCurveRNN
from data.dataset import LightCurveDataset
from torch.utils.data import DataLoader

# Load dataset
dataset = LightCurveDataset('data/processed_training.csv', max_length=200)
loader = DataLoader(dataset, batch_size=32, shuffle=False)

# Load model
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = LightCurveRNN(input_size=1, hidden_size=128, num_layers=3, num_classes=dataset.num_classes).to(device)
model.load_state_dict(torch.load('rnn_model.pth', map_location=device))
model.eval()

# Test on a few samples
batch_data, batch_labels = next(iter(loader))
batch_data = batch_data.to(device)
with torch.no_grad():
    outputs = model(batch_data)
    predictions = outputs.argmax(dim=1)
    
print(f"Sample predictions: {predictions[:10].cpu().numpy()}")
print(f"Sample labels: {batch_labels[:10].cpu().numpy()}")
print(f"Accuracy on batch: {(predictions.cpu() == batch_labels).float().mean().item():.4f}")